In [1]:
from models import Statements_table_log
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from scrapping_sources.Macrotrend import Macrotrend
import datetime
import pandas as pd

In [4]:
USERNAME='postgres'
HOST='database-webscrap.ccaaerr6cq44.eu-west-3.rds.amazonaws.com'
DATABASE_NAME='webscrapping'
PASSWORD='postgres5678'
PORT=5432
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

session = sessionmaker(bind=create_engine(DATABASE_URI))
s = session()
M = Macrotrend()
s.close_all()

/tmp/ipykernel_318036/1296398744.py:11: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [74]:
import pandas as pd
import requests, json, re
from bs4 import BeautifulSoup as bs
import datetime
import concurrent.futures
from itertools import repeat

class Macrotrend():
        
    def get_statement(self, ticker, statement, time_format):

        statement_data = None #default load

        if time_format == 'annual':

            try:
                r = requests.get('https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data
        
            except:
                print(f"No data available for {ticker}, {statement}, annual")
                pass
            # pass
        else:

            try:

                r = requests.get(f'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                company_name = r.url.split('/')[6]
                r = requests.get(f'https://www.macrotrends.net/stocks/charts/'+ ticker + '/' + company_name + '/' + statement + '?freq=Q')
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data

            except:

                print(f"No data available for {ticker}, {statement}, quarterly")

                pass

    def move_column(self, df, column, pos):

        col = df[column]
        df.drop(columns=[column],inplace = True)
        df.insert(pos, column, col)

        return df
    
    def generate_statement_key(self, statement, time_format):

        report_formats  = {'quarterly'  : 'Q',
                            'annual'    : 'A'}
        statements      = {'income-statement'       :'IS',
                            'balance-sheet'         :'BS',
                            'cash-flow-statement'   :'CF',
                            'financial-ratios'      :'R'}

        statement_key = statements[statement] + '-' + report_formats[time_format]

        return statement_key

    def arrange_data(self, ticker, statement, time_format):

        data_dict = []
        report_formats = {'quarterly' : 'Q',
                        'annual': 'A'}
        statements = {'income-statement':'IS',
                    'balance-sheet':'BS',
                    'cash-flow-statement':'CF',
                    'financial-ratios':'R'}

        df = self.get_statement(ticker, statement, time_format)

        if isinstance(df, pd.DataFrame):

            for i in df.columns[1:]:
                keys = df[df.columns[0]]
                data = df[i].values
                data = {k:v for k,v in zip(keys, data)}
                date = i
                data['date'] = date
                data['ticker'] = ticker
                data['statement_format'] = statements[statement] + '-' + report_formats[time_format]
                data_dict.append(data)

            df = pd.DataFrame(data_dict)
            df = self.move_column(df, 'date', 0)
            df = self.move_column(df, 'ticker', 1)
            df = self.move_column(df, 'statement_format', 2)

            my_series = []
            line_item = []
            amount = []
            date = []
            ticker = []
            security_id = []
            statement_format = []

            for i in df.iterrows():

                serie = pd.Series(i)[1]
                my_series.append(serie)
                [statement_format.append(serie[2]) for i in range(len(serie.index.values[4:]))]
                # [security_id.append(serie[3]) for i in range(len(serie.index.values[4:]))]
                [line_item.append(i) for i in  serie.index.values[4:]]
                [amount.append(i) for i in serie.values[4:]]
                [date.append(serie[0]) for i in  range(len(serie.index.values[4:]))]
                [ticker.append(serie[1]) for i in range(len(serie.index.values[4:]))]

            data = pd.DataFrame([date, statement_format, ticker, security_id,line_item, amount]).T
            data.columns = ['date','statement','ticker','security_id','line_item','amount']

            return data

        else:
            pass

    def generate_statement_table_multi(self, ticker_list, statement, time_format):

        table   = []

        for i in ticker_list:

            df = self.arrange_data(i, statement, time_format)

            if isinstance(df, pd.DataFrame):
                table.append(df)
            else:
                pass

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        return table_concat

    def generate_statement_table_multi_threading(self, ticker_list, statement, time_format):

        table = []

        def create_table(ticker, statement, time_format):

            df = self.arrange_data(ticker, statement, time_format)
            if isinstance(df, pd.DataFrame):
                table.append(df)

        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            executor.map(create_table, ticker_list, repeat(statement), repeat(time_format))

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])

        return table_concat
    
    def latest_ending_period_available(self, ticker, statement='balance-sheet', time_format='quarterly'):

        try:
            df = self.get_statement(ticker, statement, time_format)
            raw_date = df.columns[1]
            converted_date = datetime.datetime.strptime(str(raw_date), '%Y-%m-%d').strftime("%Y-%m")
        except:
            converted_date = None

        return converted_date
    

In [75]:
from models import Security
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)

s = Session()
r = s.query(Security.ticker, Security.id).all()
sec_id_map = {i[0]:i[1] for i in r}
# df['security_id'] = df.ticker.map(sec_id_map)
# df[df['date','ticker', 'statement']]
# sec_id_map

In [76]:
def generate_statement_list(ticker, statement, time_format):

    df = M.arrange_data(ticker, statement, time_format)
    df = df[['date','ticker', 'statement', 'security_id']].drop_duplicates()
    df["security_id"] = df["ticker"].map(sec_id_map)
    # Getting there, but what's next?...
    # 1 - Create a statement_id
    # 
    return df

In [77]:
df["statement"][(df['date'] == '2021-06-30') & (df['ticker'] == 'MSFT')].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [78]:
def retrieve_statement_id(date, ticker):
    # df should be generated from existing listing database

    return df["statement"][(df['date'] == date) & (df['ticker'] == ticker)].values[0]

In [79]:
retrieve_statement_id('2021-06-30', 'MSFT')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [80]:
df['d'] = df.apply(lambda x: retrieve_statement_id(date = x['date'], ticker= x['ticker']), axis=1)

In [81]:
df

,date,ticker,statement,security_id,d
0,2021-07-31,NAPA,R-Q,5068,R-Q
19,2021-04-30,NAPA,R-Q,5068,R-Q
38,2020-07-31,NAPA,R-Q,5068,R-Q
57,2020-04-30,NAPA,R-Q,5068,R-Q
76,2019-07-31,NAPA,R-Q,5068,R-Q


In [82]:
from models import Statements_table_log

# TO UPDATE THE STATEMENT LIST, LET'S USE THE GENERATE STATEMENT LIST
#  ON THESE TICKERS AND APPLY A SET REDUCTION
def get_todays_statements_update():

    r = s.query(
        Statements_table_log.ticker,
        Statements_table_log.time_format,
        Statements_table_log.statement,
        Statements_table_log.date,
    ).where(
        Statements_table_log.date == datetime.date.today(),
        Statements_table_log.status == 'updated'
    ).all()

    return pd.DataFrame(r)

In [83]:
df = get_todays_statements_update()
df

for row in df.iterrows():
    ticker = row[1][0]
    stmnt = row[1][2]
    t_format = row[1][1]
    

    print(ticker, stmnt, t_format,)

In [84]:
df_g = generate_statement_list('CMTL', 'balance-sheet', 'quarterly')

CMTL <Response [200]>


In [85]:
get_statement_list_db('NAPA')

,date,ticker,statement,security_id
0,2021-07-31,NAPA,IS-A,5068
1,2020-07-31,NAPA,IS-A,5068
2,2019-07-31,NAPA,IS-A,5068
3,2018-07-31,NAPA,IS-A,5068
4,2021-07-31,NAPA,CF-Q,5068
5,2021-07-31,NAPA,BS-A,5068
6,2020-07-31,NAPA,BS-A,5068
7,2019-07-31,NAPA,BS-A,5068
8,2018-07-31,NAPA,BS-A,5068
9,2021-07-31,NAPA,BS-Q,5068


In [93]:
from models import Statements_list_table

def get_statement_list_db(ticker):
    """Returns a dataframe containing all statements of a particular ticker"""
    r = s.query(
        Statements_list_table.date,
        Statements_list_table.ticker,
        Statements_list_table.statement,
        Statements_list_table.security_id,
        # Statements_list_table.statement_id
    ).where(
        Statements_list_table.ticker == ticker,
    ).all()

    df = pd.DataFrame(r)
    df.columns = [['date','ticker','statement','security_id']]

    return df

def generate_statement_list(ticker, statement, time_format):
    """Generates a statements list from macrotrend based on the arguments:
        -ticker,
        -statement,
        -time_format"""
    df = M.arrange_data(ticker, statement, time_format)
    df = df[['date','ticker', 'statement', 'security_id']].drop_duplicates()
    df["security_id"] = df["ticker"].map(sec_id_map)

    return df

statements = [
    'income-statement',
    'balance-sheet',
    'cash-flow-statement',
    'financial-ratios'
]
time_format = [
    'annual',
    'quarterly'
]

def generate_statement_list_all(ticker):
    """generates a statement list from Macrotrend with all statements
    and time formats combined, for a ticker"""
    table = []

    for stmnt in statements:
        for t_format in time_format:

            df = generate_statement_list(ticker, stmnt, t_format)
            table.append(df)

    results = pd.concat(table)
    results.columns = [['date','ticker','statement','security_id']]

    return results

In [104]:
db =  get_statement_list_db('NAPA')
m = generate_statement_list_all('NAPA')
update = pd.concat([m,db]).drop_duplicates(keep=False)

OperationalError: (psycopg2.errors.AdminShutdown) terminating connection due to administrator command
SSL connection has been closed unexpectedly

[SQL: SELECT statements_list_table.date AS statements_list_table_date, statements_list_table.ticker AS statements_list_table_ticker, statements_list_table.statement AS statements_list_table_statement, statements_list_table.security_id AS statements_list_table_security_id 
FROM statements_list_table 
WHERE statements_list_table.ticker = %(ticker_1)s]
[parameters: {'ticker_1': 'NAPA'}]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

,date,ticker,statement,security_id


In [102]:
db.shape

(36, 4)